##SETUP

In [1]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)


SLEEP_DELAY=0.1

# Start Scrcpy (once after PC restart)
start_scrcpy=False
if start_scrcpy:
    proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
    time.sleep(6) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process


bot = bot_core.Bot()

In [2]:
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

## Game stats

In [159]:
# get stored mana 
#bot.getText(220,1360,90,50,new=False,digits=True)

# get unit mana 
bot.getText(450,1360,90,50,new=False,digits=True)

# Get upgrade mana
#bot.getText(20,1540,700,50,new=False,digits=True).split('6')


'70'

In [3]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
grid_df=bot_perception.grid_status(names,grid_df)
display(grid_df)
prev_grid= grid_df.copy()
#cache_grid= grid_df.copy()

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,empty.png,999.0,0,0.0,"[0, 0]",1.0
1,icon_1,alchemist.png,700.0,1,16184.5,"[0, 1]",1.0
2,icon_2,sharpshooter.png,335.0,2,940.0,"[0, 2]",1.0
3,icon_3,alchemist.png,228.0,1,1577.5,"[0, 3]",1.0
4,icon_4,vampire.png,187.0,1,1459.5,"[0, 4]",1.0
5,icon_5,empty.png,999.0,0,0.0,"[1, 0]",1.0
6,icon_6,empty.png,999.0,0,0.0,"[1, 1]",1.0
7,icon_7,vampire.png,179.0,1,4235.5,"[1, 2]",1.0
8,icon_8,lightning.png,190.0,1,2373.0,"[1, 3]",1.0
9,icon_9,lightning.png,178.0,1,1476.5,"[1, 4]",1.0


In [709]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
group_keys.remove(('empty.png',0))
#unit_series.drop(labels='empty.png',level=0)
merge_series = unit_series[unit_series>2]
merge_target =  merge_series.sample().index[0]
merge_df=df_split.get_group(merge_target)
merge_df.sample(n=2)

,grid_id,unit,probability,rank,rank_error,position,Age
12,icon_12,priest.png,175.0,1,2068.5,"[2, 2]",0.0
13,icon_13,priest.png,200.0,1,1269.5,"[2, 3]",4.0


# Play Game

In [4]:
bot.getScreen()

In [3]:
bot.restart_RR()

## Extra unit logic 

In [41]:
##Extra unit logic 
# Try to merge any priestess
#if 'priest.png' in unit_list:
#    if df_groups['priest.png']<1:
#        bot.merge_unit(grid_df,'priest.png')

# Basic Loop

In [2]:
def try_merge(rank=1,prev_grid=None):
    info=''
    merge_df =None
    names=bot.scan_grid(new=True)
    grid_df=bot_perception.grid_status(names,prev_grid=prev_grid)
    df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
    # Select stuff to merge
    merge_series = unit_series[unit_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
    # check if grid full
    if ('empty.png',0) in group_keys:
        # Try to merge priest
        merge_priest = bot_core.filter_keys(merge_series,['priest.png'])
        if not merge_priest.empty:
            merge_df = bot.merge_unit(df_split,merge_priest)
        # Merge if full board
        if df_groups['empty.png']<=2:
            info='Merging!'
            # Add criteria
            merge_series = bot_core.filter_keys(merge_series,[rank,'priest.png'])
            if not merge_series.empty:
                # drop duplicated indices
                merge_series = merge_series[~merge_series.index.duplicated()]
                # Take index name of random sample
                merge_target =  merge_series.sample().index[0] 
                # Retrieve group    
                merge_df=df_split.get_group(merge_target)
                #merge_df=merge_df.sort_values(by='Age',ascending=False).reset_index(drop=True)
                # Send merge command
                merge_df = bot.merge_unit(df_split,merge_series)
            else:
                info='not enough filtered targets!'
        else: info= 'need more units!'
    # If grid seems full, merge any
    else:
        info = 'Full Grid - Merging!'
        merge_df = bot.merge_unit(df_split,merge_series)
    return grid_df,unit_series,merge_df,info
    
def get_button_pos(df,button):
    button=button+'.png'
    pos=df[df['icon']==button]['pos [X,Y]'].reset_index(drop=True)[0]
    return pos
# Mana level cards
def mana_level(cards, hero_power=False):
    upgrade_pos_dict={
    1:[100,1500], 2:[200,1500],
    3:[350,1500], 4:[500,1500],
    5:[650,1500] }
    # Level each card
    for card in cards:
        bot.click(*upgrade_pos_dict[card])
    if hero_power:
        bot.click(800,1500)

def battle_screen(start=False):
    # Scan screen for any key buttons
    button_df = bot.get_current_icons()
    avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)
    if not avail_buttons.empty:
        # list of buttons
        button_names=avail_buttons['icon'].tolist()
        if 'fighting.png' in button_names and not '0cont_button.png' in button_names:
            return avail_buttons,'fighting'
        # Start pvp if homescreen
        if start and 'pvp_button.png' in button_names and 'battle_icon.png' in button_names:
            pvp_pos = get_button_pos(button_df,'pvp_button')
            bot.click_button(pvp_pos)
            time.sleep(1)
            return avail_buttons, 'home'
        # Check first button is clickable
        first_button=button_names[0].split('.')[0]
        if first_button in ['back_button','battle_icon','0cont_button','1quit']:
            button_pos=avail_buttons['pos [X,Y]'].tolist()[0]
            bot.click_button(button_pos)
            return avail_buttons,'menu'
    else:
        bot.shell(f'input keyevent {const.KEYCODE_BACK}') #Force back
        return button_df,'lost'
        # add recursive call to restart last recorse?
battle_screen()

(              icon  available    pos [X,Y]
 0  battle_icon.png       True  (359, 1419)
 1   pvp_button.png       True  (140, 1259),
 'menu')

In [ ]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
# Select stuff to merge
merge_series = unit_series[unit_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
# check if grid full
if ('empty.png',0) in group_keys:
    # Try to merge priest
    merge_priest = bot_core.filter_keys(merge_series,['priest.png'])

In [13]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
# Select stuff to merge
merge_series = unit_series[unit_series>=2] # at least 2 units 
merge_series

unit           rank
lightning.png  1       3
alchemist.png  1       2
empty.png      0       2
priest.png     2       2
Name: unit, dtype: int64

In [14]:
wait=0
combat = 0
grid_df =None
# Main loop
while(1):
    output = battle_screen(start=True)
    if output[1]=='fighting':
        wait = 0
        combat+=1
        if combat>20:
            bot.restart_RR()
            combat = 0
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Spawn units
                bot.click(450,1360)
                # Upgrade units
                mana_level([5,2,3,4],hero_power=True)
                # Try to merge units
                grid_df,unit_series,df_groups,info = try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 '+output[1] +'  '+info,grid_df,unit_series[0:5],df_groups])
    else:
        combat=0
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR()
            wait=0
        print(output[1],'wait count:',wait)

lost wait count: 1
lost wait count: 2
lost wait count: 3
lost wait count: 4
lost wait count: 5
lost wait count: 6
lost wait count: 7
lost wait count: 8
lost wait count: 9
lost wait count: 10
lost wait count: 11
lost wait count: 12
lost wait count: 13
lost wait count: 14
lost wait count: 15
lost wait count: 16
lost wait count: 17
lost wait count: 18
lost wait count: 19
lost wait count: 20
lost wait count: 21
lost wait count: 22
lost wait count: 23
lost wait count: 24
lost wait count: 25
lost wait count: 26
lost wait count: 27
lost wait count: 28
lost wait count: 29
lost wait count: 30
lost wait count: 31
lost wait count: 32
lost wait count: 33
lost wait count: 34
lost wait count: 35
lost wait count: 36
lost wait count: 37
lost wait count: 38
lost wait count: 39
lost wait count: 40
RESTARTING
lost wait count: 0
lost wait count: 1
lost wait count: 2
lost wait count: 3
lost wait count: 4
lost wait count: 5
lost wait count: 6
lost wait count: 7
lost wait count: 8
lost wait count: 9
lost wai

KeyboardInterrupt: 

In [25]:
    def restart_RR(quick_disconnect=False):
            if quick_disconnect:
                for i in range(40):
                    bot.shell(f'input keyevent {const.KEYCODE_APP_SWITCH}') #Go app switch
                    return
            # Force kill game through ADB shell
            bot.shell('am force-stop com.my.defense')
            time.sleep(2)
            # Launch application through ADB shell
            bot.shell('monkey -p com.my.defense 1')
            time.sleep(10) # wait for app to load

restart_RR()


In [36]:
bot.restart_RR()